# NoteBook To Check Point or Polygon Inside Polygon

The objective of this notebook is to search a set of points or polygons for spatial overlap with a largerset

By Cascsde Tuholske 2019-01-17

#### Notes
1. Points should be dialated to circles to prevent one falling just outside the larger polygon set (see Lusaka)

### Depedencies

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping
import rasterio
import copy

### Functions

In [2]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [3]:
def point_buffer(gpd_df, raduis):
    "Function to make a shapely polygon buffer around a point"
   
    # AGU 2018-12-04 - radius set to ~250m at the equator 
    #radius = 250*1/(111*1000)

    # new_gpd_df = gpd.GeoDataFrame()
    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [4]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Note 2018-11-30 update arr to gpd_df 

In [5]:
def poly_overlap (point_buffer, poly_raster):
    """
    This function will check if point buffers intersect with polygons 
    if given two gpd dataframes with point buffers and polygons
    Returns point ids, point geometry, polygon index #, and polygon geometry in a geopandas DF.
    It goes faster if smaller list goes first
    
    Change TOWN & CITY ARRA
    """
    
    # make arrays to fill
    country_arr = []
    city_arr = []
    osm_type_arr = []
    lat_arr = []
    lon_arr = []
    osm_id_arr = [] 
    FID_arr = [] 
    poly_geom_arr = []
    #point_geom_arr = []
    
    for index_point_buffer, row_point_buffer in point_buffer.iterrows():
        for index_poly_raster, row_poly_raster in poly_raster.iterrows():
            if row_point_buffer['geometry'].intersects(row_poly_raster['geometry']):
                country = row_point_buffer['country']
                city = row_point_buffer['town'] # Need to change town vs city
                osm_type = row_point_buffer['osm_type']
                lat = row_point_buffer['lat']
                lon = row_point_buffer['lon']
                osm_id = row_point_buffer['osm_id']
                poly_id = row_poly_raster['FID']
                poly_geom = shape(mapping(row_poly_raster['geometry'])) # make polygon
                #point_geom = row_point_buffer['old_geom']

                country_arr.append((country))
                city_arr.append((city))
                osm_type_arr.append((osm_type))
                lat_arr.append((lat))
                lon_arr.append((lon))
                osm_id_arr.append((osm_id))
                FID_arr.append((poly_id))
                poly_geom_arr.append((poly_geom))
                #point_geom_arr.append((point_geom))
    
    # put results into a geopandas df
    new_gpd_df = gpd.GeoDataFrame()
    new_gpd_df['osm_id'] = osm_id_arr
    new_gpd_df['FID'] = FID_arr
    new_gpd_df['geometry'] = poly_geom_arr
    new_gpd_df['country'] = country_arr           
    new_gpd_df['city'] = city_arr
    new_gpd_df['osm_type'] = osm_type_arr
    new_gpd_df['lat'] = lat_arr
    new_gpd_df['lon'] = lon_arr
    #new_gpd_df['point_geom'] = point_geom_arr
    
    
    return new_gpd_df

# Note 2018-11-30 update arr to gpd_df 

# OSM Merge Cities and Towns

In [6]:
temp_data = '/Users/cascade/Github/Pop-ERL/temp_data/ERL19/'
towns = '20190430_osm_DRC_towns.csv'
cities = '20190430_osm_DRC_cities.csv'
erl_v2_data = '/Users/cascade/Github/Pop-ERL/temp_data/ERL19/ERLv2/'

towns_gdf = load_points(erl_v2_data+towns)
cities_gdf = load_points(erl_v2_data+cities)

towns_gdf['osm_type'] = 'town'
cities_gdf['osm_type'] = 'city'

print(len(towns_gdf))
print(len(cities_gdf))

osm_points = pd.concat([towns_gdf, cities_gdf], ignore_index=True, sort = False)
print(len(osm_points))

495
56
551


In [7]:
osm_points.tail()

,country,osm_id,town,lat,lon,geometry,osm_type
546,Democratic Republic of the Congo,3679354870,Lubutu,-0.740336,26.581553,POINT (26.5815525 -0.7403356),city
547,Democratic Republic of the Congo,4713759630,Kirumba,-0.559707,29.188568,POINT (29.1885684 -0.5597065999999999),city
548,Democratic Republic of the Congo,4853946100,Likati,3.362353,23.888545,POINT (23.8885454 3.362353),city
549,Democratic Republic of the Congo,4882355591,Kikwit,-5.038308,18.817845,POINT (18.8178447 -5.0383083),city
550,Democratic Republic of the Congo,4882355592,Matadi,-5.825656,13.460905,POINT (13.4609047 -5.8256559),city


In [8]:
# AGU 2018-12-04 - radius set to ~250m at the equator 

radius = 250*1/(111*1000)
radius

0.0022522522522522522

In [9]:
osm_buffer_gdf = point_buffer(osm_points, radius)

In [10]:
osm_buffer_gdf.head(6)

,country,osm_id,town,lat,lon,old_geom,osm_type,geometry
0,Democratic Republic of the Congo,27043347,Barumbu,-4.316827,15.326931,POINT (15.3269312 -4.316827200000001),town,POLYGON ((15.32918345225225 -4.316827200000001...
1,Democratic Republic of the Congo,27043358,Lingwala,-4.328649,15.301997,POINT (15.3019974 -4.3286488),town,"POLYGON ((15.30424965225225 -4.3286488, 15.304..."
2,Democratic Republic of the Congo,27043362,Kintambo,-4.339552,15.266192,POINT (15.266192 -4.339552200000001),town,POLYGON ((15.26844425225225 -4.339552200000001...
3,Democratic Republic of the Congo,27043390,Kasa-Vubu,-4.346474,15.304893,POINT (15.304893 -4.346474099999999),town,POLYGON ((15.30714525225225 -4.346474099999999...
4,Democratic Republic of the Congo,29804794,Kasangulu,-4.590024,15.167657,POINT (15.1676574 -4.590024400000001),town,POLYGON ((15.16990965225225 -4.590024400000001...
5,Democratic Republic of the Congo,29806455,Ndjili,-4.400306,15.371449,POINT (15.3714491 -4.400306099999999),town,POLYGON ((15.37370135225225 -4.400306099999999...


In [11]:
osm_buffer_gdf = osm_buffer_gdf.drop(['old_geom'], axis=1)
osm_buffer_gdf.head(6)

,country,osm_id,town,lat,lon,osm_type,geometry
0,Democratic Republic of the Congo,27043347,Barumbu,-4.316827,15.326931,town,POLYGON ((15.32918345225225 -4.316827200000001...
1,Democratic Republic of the Congo,27043358,Lingwala,-4.328649,15.301997,town,"POLYGON ((15.30424965225225 -4.3286488, 15.304..."
2,Democratic Republic of the Congo,27043362,Kintambo,-4.339552,15.266192,town,POLYGON ((15.26844425225225 -4.339552200000001...
3,Democratic Republic of the Congo,27043390,Kasa-Vubu,-4.346474,15.304893,town,POLYGON ((15.30714525225225 -4.346474099999999...
4,Democratic Republic of the Congo,29804794,Kasangulu,-4.590024,15.167657,town,POLYGON ((15.16990965225225 -4.590024400000001...
5,Democratic Republic of the Congo,29806455,Ndjili,-4.400306,15.371449,town,POLYGON ((15.37370135225225 -4.400306099999999...


### Analysis

In [34]:
# Load Polygons

polygons = 'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300'

raster_polygons = gpd.read_file(temp_data+polygons+'.shp')
print(len(raster_polygons))

45281


In [35]:
raster_polygons.head(6)

,DN,geometry
0,1,"POLYGON ((814405.4525564685 4480029.461868489,..."
1,1,"POLYGON ((854405.4525564685 4490029.461868489,..."
2,1,"POLYGON ((873405.4525564685 4480029.461868489,..."
3,1,"POLYGON ((881405.4525564685 4478029.461868489,..."
4,1,"POLYGON ((865405.4525564685 4475029.461868489,..."
5,1,"POLYGON ((884405.4525564685 4476029.461868489,..."


### Change CRS if needed

In [36]:
raster_polygons.crs

{'proj': 'moll',
 'lon_0': 0,
 'x_0': 0,
 'y_0': 0,
 'ellps': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [37]:
osm_buffer_gdf.crs

{'init': 'epsg:4326'}

In [38]:
# For GHS CRS

crs = {'ellps': 'WGS84',
 'lon_0': 0,
 'no_defs': True,
 'proj': 'moll',
 'units': 'm',
 'x_0': 0,
 'y_0': 0}

In [39]:
raster_polygons.crs = crs

In [40]:
raster_polygons_crs = raster_polygons.to_crs(osm_buffer_gdf.crs)

In [41]:
# For GHS make FID and reset crs
FID = list(range(len(raster_polygons)))
raster_polygons['DN'] = FID
raster_polygons.columns.values[0] = "FID"

In [42]:
raster_polygons.head()

,FID,geometry
0,0,"POLYGON ((814405.4525564685 4480029.461868489,..."
1,1,"POLYGON ((854405.4525564685 4490029.461868489,..."
2,2,"POLYGON ((873405.4525564685 4480029.461868489,..."
3,3,"POLYGON ((881405.4525564685 4478029.461868489,..."
4,4,"POLYGON ((865405.4525564685 4475029.461868489,..."


### Current Start 
1. Data: 2019-01-18 @ 6:22 pm
2. Raster Poly File: GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300.shp
3. OSM Town File: 20190114_osm_africa_towns.csv
4. Clip: [25000:35000]
5. outfile - GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_subB1.shp

In [30]:
# run poly_overlap

import time
checkpoint = time.time()

poly_overlap_out = poly_overlap(osm_buffer_gdf, raster_polygons)

print("elapsed time is: {}s".format(time.time()-checkpoint))
print(len(poly_overlap_out))

elapsed time is: 37.698596715927124s
133


In [31]:
print(len(poly_overlap_out))

133


In [32]:
poly_overlap_out.tail(20)

,osm_id,FID,geometry,country,city,osm_type,lat,lon
113,435755862,915,"POLYGON ((25.532916 -10.742083, 25.532916 -10....",Democratic Republic of the Congo,Kolwezi,city,-10.716995,25.466992
114,435789385,890,"POLYGON ((26.732916 -11.00875, 26.741249 -11.0...",Democratic Republic of the Congo,Likasi,city,-10.989160,26.739742
115,435818462,1444,"POLYGON ((29.499582 0.5912500000000001, 29.516...",Democratic Republic of the Congo,Beni,city,0.493934,29.465322
116,435856567,1805,"POLYGON ((19.782916 1.224584, 19.791249 1.2245...",Democratic Republic of the Congo,Basankusu,city,1.219168,19.801370
117,435884993,1360,"POLYGON ((29.132916 -3.425416, 29.132916 -3.43...",Democratic Republic of the Congo,Uvira,city,-3.405587,29.137551
118,436003765,982,"POLYGON ((24.999582 -8.70875, 25.016249 -8.708...",Democratic Republic of the Congo,Kamina,city,-8.735183,25.003220
119,436044301,1180,"POLYGON ((29.507916 -3.892083, 29.524582 -3.89...",Democratic Republic of the Congo,Bukavu,city,-2.505584,28.859489
120,478528026,1788,"POLYGON ((27.591249 2.774584, 27.599582 2.7745...",Democratic Republic of the Congo,Isiro,city,2.774270,27.620817
121,612128950,1749,"POLYGON ((18.641249 3.649584, 18.641249 3.6329...",Democratic Republic of the Congo,Libenge,city,3.652938,18.635736
122,615209242,1042,"POLYGON ((20.807916 -6.392083, 20.832916 -6.39...",Democratic Republic of the Congo,Tshikapa,city,-6.423050,20.788769


In [33]:
poly_overlap_out.to_file(erl_v2_data+polygons+'_polyoverlap.shp', driver='ESRI Shapefile')